## Import

In [ ]:
import os
import pathlib
import sys

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

sys.path.append(pathlib.Path.cwd().parent.as_posix())
import tensorflow as tf
from keras.utils.layer_utils import count_params

from src.models.lib.base import base_unet_pp
from src.models.lib.builder import build_unet_pp
from src.models.lib.config import UNetPPConfig
from src.models.lib.loss import log_cosh_loss
from src.models.lib.utils import loss_dict_gen
from src.models.train_model import create_dataset

## Model Initialization



In [ ]:
## Custom Model Initialization
## Used for debugging

custom_mobile_unet_pp_param = UNetPPConfig(
    model_name="test_custom",
    upsample_mode="upsample",
    depth=3,
    input_dim=(512, 512, 1),
    batch_norm=True,
    model_mode="mobile",
    n_class={"bin": 1, "mult": 4},
    deep_supervision=False,
    filter_list=[2, 2, 2],
    downsample_iteration=[1, 1, 1],
)

custom_mobile_unet, custom_output_layers_name = build_unet_pp(
    custom_mobile_unet_pp_param, custom=True
)
count_params(custom_mobile_unet.trainable_weights)

## Create Dataset

In [ ]:
project_root = pathlib.Path.cwd().parent
coca_dataset = create_dataset(
    project_root, custom_mobile_unet_pp_param, custom_output_layers_name, batch_size=1
)

In [ ]:
train_coca_dataset = coca_dataset["train"]
val_coca_dataset = coca_dataset["val"]

## Model Compilation & Training

In [ ]:
loss_dict = loss_dict_gen(
    custom_mobile_unet_pp_param,
    custom_output_layers_name,
    [log_cosh_loss, "categorical_crossentropy"],
)

custom_mobile_unet.compile(optimizer="adam", loss=loss_dict)

In [ ]:
custom_mobile_unet.fit(
    train_coca_dataset, epochs=1, batch_size=1, validation_data=val_coca_dataset
)

In [ ]:
custom_mobile_unet.summary()